In [1]:
import nltk

In [2]:
#nltk.download_shell()

In [3]:
messages = [line.rstrip() for line in open('SMSSpamCollection')]
#printing total number of messages
print(len(messages))

5574


In [4]:
#print first 50 messages
for mess_no, message in enumerate(messages[:50]):
    print(mess_no, message)
    print('\n')

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim cod

In [5]:
import pandas as pd

In [6]:
# used to_csv function to read a tsv file and convert it to a datadrame
messages_df = pd.read_csv('SMSSpamCollection', sep='\t', names=['category', 'message'])
messages_df

,category,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
messages_df.groupby('category').describe()

message                                                            \
           count unique                                                top   
category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
category       
ham        30  
spam        4

In [8]:
messages_df['length'] = messages_df['message'].apply(len)

In [9]:
messages_df

,category,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160
5568,ham,Will ü b going to esplanade fr home?,36
5569,ham,"Pity, * was in mood for that. So...any other s...",57
5570,ham,The guy did some bitching but I acted like i'd...,125


In [10]:
messages_df.describe()

,length
count,5572.000000
mean,80.489950
std,59.942907
min,2.000000
25%,36.000000
50%,62.000000
75%,122.000000
max,910.000000


In [11]:
num = []
cat = ['category']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in cat:
    messages_df[i] =le.fit_transform(messages_df[i])

In [12]:
import string
from nltk.corpus import stopwords

In [13]:
def text_process(mess):
    """
    Tokenization:
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [14]:
messages_df['message'].head(5).apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages_df['message'])
messages_df_bow = bow_transformer.transform(messages_df['message'])
print(len(bow_transformer.vocabulary_))

11425


In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_df_bow)
messages_df_tfidf = tfidf_transformer.transform(messages_df_bow)

In [17]:
from sklearn.naive_bayes import MultinomialNB
spam_detection = MultinomialNB().fit(messages_df_tfidf, messages_df['category'])

In [18]:
from sklearn.model_selection import train_test_split
x = messages_df['message']
y = messages_df['category']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [19]:
# naive bayes
from sklearn.naive_bayes import MultinomialNB
spam_detection = MultinomialNB().fit(messages_df_tfidf, messages_df['category'])

from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [20]:
#KNN Classifier
#from sklearn.neighbors import KNeighborsClassifier
#classifier = KNeighborsClassifier(n_neighbors=61, p=2,metric='euclidean')
#from sklearn.pipeline import Pipeline

#pipeline = Pipeline([
#    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
#    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
#    ('classifier', KNeighborsClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
#])

In [21]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x0000022BA9C9A310>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [22]:
pred = pipeline.predict(x_test)
pred

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [24]:
cm = confusion_matrix(y_test, pred)
print (cm)
print(accuracy_score(y_test, pred))
print(precision_score(y_test, pred))
print(f1_score(y_test, pred))

[[1455    0]
 [  59  158]]
0.9647129186602871
1.0
0.8426666666666667


In [25]:
from sklearn.metrics import classification_report
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1514
           1       0.73      1.00      0.84       158

    accuracy                           0.96      1672
   macro avg       0.86      0.98      0.91      1672
weighted avg       0.97      0.96      0.97      1672

